In [38]:
import json
import os
import pandas as pd
from pprint import pprint

In [76]:
__path__ = "memory"
latest_file = sorted(os.listdir(__path__))[-1]
print(latest_file)
with open(os.path.join(__path__, latest_file), 'r') as f:
        data = json.load(f)

2025-06-24T09-19-27-409Z_run.json


In [77]:
data["sessions"]

{'b94c8c37-0e44-4017-a3ee-c5a4d0288c9f': {'userId': 104,
  'condition': 1,
  'usesEmotion': False,
  'sessionId': 'b94c8c37-0e44-4017-a3ee-c5a4d0288c9f',
  'createdAt': '2025-06-24T09:20:09.120Z',
  'lastActive': '2025-06-24T09:23:26.905Z'},
 '1d2f9eec-bbc4-4205-8923-2da155924ae9': {'userId': 104,
  'condition': 0,
  'usesEmotion': False,
  'sessionId': '1d2f9eec-bbc4-4205-8923-2da155924ae9',
  'createdAt': '2025-06-24T09:23:50.922Z',
  'lastActive': '2025-06-24T09:25:42.924Z'}}

In [85]:
session_of_interest = ""
for key, value in data["sessions"].items():
    if value["condition"] == 1: session_of_interest = key #1 is with  emotion

print(f"Session of interest: {session_of_interest}")

def get_filtered_data(data, session_of_interest):
    for_df = {"sessionId": [], "role": [], "message": [], "positive": [], "negative": [], "neutral": []}
    for key, value in data["messages"].items():
        if value["sessionId"] == session_of_interest:
            for_df["sessionId"].append(value["sessionId"])
            for_df["role"].append(value["role"])
            for_df["message"].append(value["content"])
            emo = value["mergedSentiment"]
            if emo == "positive":
                for_df["positive"].append(1)
                for_df["negative"].append(0)
                for_df["neutral"].append(0)
            elif emo == "negative":
                for_df["positive"].append(0)
                for_df["negative"].append(1)
                for_df["neutral"].append(0)
            elif emo == "neutral":  
                for_df["positive"].append(0)
                for_df["negative"].append(0)
                for_df["neutral"].append(1)
            else:
                #print(f"Unknown sentiment: {emo}")
                for_df["positive"].append(0)
                for_df["negative"].append(0)
                for_df["neutral"].append(0)

    df = pd.DataFrame(for_df)
    df["Important Tutor Turn"] = ["No"]*len(df)

    return pd.DataFrame(df)

Session of interest: b94c8c37-0e44-4017-a3ee-c5a4d0288c9f


In [91]:
df = get_filtered_data(data, session_of_interest)
dest_dir = "emo_surveys/"
if not os.path.exists(dest_dir):
    os.mkdir(dest_dir)
df.to_excel(os.path.join(dest_dir, latest_file.split(".json")[0]+".xlsx"), index=False)
print("Open the file: ", os.path.join(dest_dir, latest_file.split(".json")[0]+".xlsx"))
df

Open the file:  emo_surveys/2025-06-24T09-19-27-409Z_run.xlsx


,sessionId,role,message,positive,negative,neutral,Important Tutor Turn
0,b94c8c37-0e44-4017-a3ee-c5a4d0288c9f,tutor,Hi there! 👋 I am your MathBuddy tutor. Let's w...,0,0,0,No
1,b94c8c37-0e44-4017-a3ee-c5a4d0288c9f,student,"Okay, total number of muffins is 10. So 3/10+3...",0,1,0,No
2,b94c8c37-0e44-4017-a3ee-c5a4d0288c9f,tutor,I see you've made a great start by figuring ou...,0,0,0,No
3,b94c8c37-0e44-4017-a3ee-c5a4d0288c9f,student,"So, 2 ways of picking from 10 muffins and I ca...",0,1,0,No
4,b94c8c37-0e44-4017-a3ee-c5a4d0288c9f,tutor,You're definitely on the right track thinking ...,0,0,0,No
5,b94c8c37-0e44-4017-a3ee-c5a4d0288c9f,student,"Aaaaaah, I don't know.",0,1,0,No
6,b94c8c37-0e44-4017-a3ee-c5a4d0288c9f,tutor,"I hear your frustration, and it's completely o...",0,0,0,No
